Train a neural network on to predict movie likeability based on text reviews.

![](images/pos_neg_sentiment.png)

Adapted from: 
- https://github.com/fchollet/keras/blob/master/examples/reuters_mlp.py
- http://machinelearningmastery.com/predict-sentiment-movie-reviews-using-deep-learning/

In [17]:
reset -fs

In [ ]:
# %load environment.yml
# Instructions to run:
# 1. cd to folder containing this file
# 2. $ conda env create --force
# 3. $ source activate dl

name: dl
channels:
  - conda-forge
  - damianavila82
dependencies:
  - python == 3.6
  - jupyter
  - rise
  - pip:
    - tensorflow == 1.0
    - keras == 2.0


In [18]:
from keras.datasets import imdb

The dataset is the [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/), often referred to as the IMDB dataset.

![](https://kaggle2.blob.core.windows.net/competitions/inclass/4996/media/Screen%20Shot%202016-02-23%20at%2010.56.44%20AM.png)

Dataset of 25,000 movies reviews from [http://www.imdb.com/](imbd.com), labeled by sentiment (positive/negative). 

>I wish i had more hands, because then I would have more thumbs, because this movie is so terrible because then i could give it so many thumbs down that thumbs down would no longer mean anything because this movie is so terrible because it sucks so badly that it made me laugh out of frustration about the story line because it just would not end because the firing and yelling just kept happening.

__Positive or Negative?__

>  Lorenzo Lamas is awesome here, and while he isn't required to act, he is quite fun to watch, and has a really cool character, and had a lot of charisma, however even he can't save this one,and he had no chemistry with the cast either! (Lamas Rules!). 

__Positive or Negative?__

Learn more: https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification

In [19]:
print('Loading data...')
num_words = 1_000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words,
                                                      skip_top=25)

print('Data loaded.')

print(f'{len(x_train):,} train sequences')
print(f'{len(x_test):,} test sequences')

Loading data...
Data loaded.
25,000 train sequences
25,000 test sequences


Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). 

For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. 

This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".

As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.

-----

![](https://www.davidtan.org/wp-content/uploads/2008/08/review-thumbs-up-down-logo-icon.jpg)

The goal is to determine whether a given movie review has a positive or negative sentiment.

In [20]:
print("Number of categories: ", len(set(y_train)))

Number of categories:  2


We will bound reviews at 500 words, truncating longer reviews and zero-padding shorter reviews.

In [21]:
from keras.preprocessing import sequence

In [22]:
max_words = 500
x_train = sequence.pad_sequences(x_train, maxlen=max_words)
x_test = sequence.pad_sequences(x_test, maxlen=max_words)

In [23]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 500)
x_test shape: (25000, 500)


-----
Now we can define a Deep Learning model
----

In [24]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Dropout, Activation, Flatten

In [25]:
print('Building model...')
model = Sequential()                         # A linear stack of layers
model.add(Embedding(input_dim=num_words,     # Input layer turns positive integers (indexes) into dense vectors of fixed size
                    output_dim= 32, 
                    input_length=max_words)) 
model.add(Flatten())                         # Change shape to make linear algebra work
model.add(Activation('relu'))                # Add non-linearity
model.add(Dropout(0.5))                      # Randomly prune so the model does not overfit
model.add(Dense(1))                          # Full connected layer
model.add(Activation('sigmoid'))             # Output layer is probability of one of two categories
print('Model built')

Building model...
Model built


In [26]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           32000     
_________________________________________________________________
flatten_2 (Flatten)          (None, 16000)             0         
_________________________________________________________________
activation_3 (Activation)    (None, 16000)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 16001     
_________________________________________________________________
activation_4 (Activation)    (None, 1)                 0         
Total params: 48,001.0
Trainable params: 48,001.0
Non-trainable params: 0.0
_________________________________________________________________


In [27]:
model.compile(loss='binary_crossentropy', # Loss measures how wrong you are; want this to be small
              optimizer='adam',           # How to search the space for best weights; Adam adapts over time
              metrics=['accuracy'])       # How to measure success

In [28]:
history = model.fit(x_train, y_train,
                    batch_size=32,
                    epochs=5,
                    verbose=True,
                    validation_split=0.1)

Train on 22500 samples, validate on 2500 samples
Epoch 1/5
22500/22500 [==============================] - 16s - loss: 0.6149 - acc: 0.6622 - val_loss: 0.4765 - val_acc: 0.8092
Epoch 2/5
22500/22500 [==============================] - 11s - loss: 0.4069 - acc: 0.8298 - val_loss: 0.3821 - val_acc: 0.8408
Epoch 3/5
22500/22500 [==============================] - 13s - loss: 0.3397 - acc: 0.8590 - val_loss: 0.3625 - val_acc: 0.8512
Epoch 4/5
22500/22500 [==============================] - 12s - loss: 0.3107 - acc: 0.8709 - val_loss: 0.3532 - val_acc: 0.8532
Epoch 5/5
22500/22500 [==============================] - 12s - loss: 0.2904 - acc: 0.8795 - val_loss: 0.3530 - val_acc: 0.8576


In [29]:
score = model.evaluate(x_test, 
                       y_test,
                       batch_size=32, 
                       verbose=True)

24640/25000 [============================>.] - ETA: 0s

In [30]:
print(f'Test accuracy: {score[1]:.2%}')

Test accuracy: 85.93%


![](https://imgs.xkcd.com/comics/machine_learning.png)

This data was collected by Stanford researchers and was used in a 2011 paper where a split of 50/50 of the data was used for training and test. An accuracy of 88.89% was achieved.

The data was also used as the basis for a Kaggle competition titled [“Bag of Words Meets Bags of Popcorn”](https://www.kaggle.com/c/word2vec-nlp-tutorial) in late 2014 to early 2015. Accuracy was achieved above 97% with winners achieving 99%.

<br>
<br> 
<br>

----